In [ ]:
# 모바일 돌린거 기준으로 이픽션넷 돌리기

In [1]:
import os
import torch

In [9]:
torch.cuda.is_available()

False

In [5]:
print(torch.version)

<module 'torch.version' from 'C:\\Users\\kyeon\\anaconda3\\Lib\\site-packages\\torch\\version.py'>


In [7]:
import torch
print(torch.__version__)

2.2.0+cpu


In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score, confusion_matrix
import seaborn as sns
import numpy as np
# 데이터 전처리
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_dir = './final'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# MobileNet 모델 불러오기 및 수정
model = models.efficientnet_b2(pretrained=False) # #@! 거꾸로
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)
# 학습 및 평가 함수
def train_model(model, criterion, optimizer, start_epoch=0, num_epochs=10):
    train_loss_history = []
    val_loss_history = []
    train_acc_history = []
    val_acc_history = []
    train_recall_history = []
    val_recall_history = []
    for epoch in range(start_epoch, start_epoch + num_epochs):
        epoch_str = f'Epoch {epoch}/{start_epoch + num_epochs - 1}'
        print(epoch_str)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0
            all_preds = []
            all_labels = []
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                # GPU 메모리 정리
                del inputs, labels, outputs, preds
                torch.cuda.empty_cache()
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_recall = recall_score(all_labels, all_preds, average='macro')
            if phase == 'train':
                train_loss_history.append(epoch_loss)
                train_acc_history.append(epoch_acc.item())
                train_recall_history.append(epoch_recall)
            else:
                val_loss_history.append(epoch_loss)
                val_acc_history.append(epoch_acc.item())
                val_recall_history.append(epoch_recall)
            log_str = f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Recall: {epoch_recall:.4f}'
            print(log_str)
        print()
    return model, train_loss_history, val_loss_history, train_acc_history, val_acc_history, train_recall_history, val_recall_history
# 하이퍼파라미터 값 범위 설정
learning_rate_range = [0.0001, 0.0002, 0.0003]
weight_decay_range = [0.00005, 0.0001, 0.002]  # 값달라서 변환
dropout_range = [0.3, 0.4, 0.5]
best_train_loss = float('inf')
best_train_accuracy = 0
best_hyperparams = {}
for lr in learning_rate_range:
    for wd in weight_decay_range:
        for dropout in dropout_range:
            print(f"Training with learning rate: {lr}, weight decay: {wd}, dropout: {dropout}")
            # 모델 초기화
            model = models.efficientnet_b2(pretrained=False) #@! #@!
            num_ftrs = model.classifier[1].in_features
            model.classifier[1] = nn.Linear(num_ftrs, len(class_names))
            model = model.to(device)
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
            criterion = nn.CrossEntropyLoss()
            model, train_loss_history, val_loss_history, train_acc_history, val_acc_history, train_recall_history, val_recall_history = train_model(
                model, criterion, optimizer, start_epoch=0, num_epochs=30  )
            avg_train_loss = sum(train_loss_history) / len(train_loss_history)
            avg_train_accuracy = sum(train_acc_history) / len(train_acc_history)
            if avg_train_loss < best_train_loss:
                best_train_loss = avg_train_loss
                best_train_accuracy = avg_train_accuracy
                best_hyperparams = {'learning_rate': lr, 'weight_decay': wd, 'dropout': dropout}
            print(f"Learning rate: {lr}, Weight decay: {wd}, Dropout: {dropout} | Avg Train Loss: {avg_train_loss:.4f} | Avg Train Accuracy: {avg_train_accuracy:.4f}")
print(f"Best hyperparameters: {best_hyperparams}")
print(f"Best train loss: {best_train_loss:.4f}, Best train accuracy: {best_train_accuracy:.4f}")
# 시각화
epochs = list(range(1, len(train_loss_history) + 1))
# 학습 및 검증 손실 시각화
plt.figure(figsize=(12, 6))
plt.plot(epochs, train_loss_history, label='Train Loss')
plt.plot(epochs, val_loss_history, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()
# 학습 및 검증 정확도 시각화
plt.figure(figsize=(12, 6))
plt.plot(epochs, train_acc_history, label='Train Accuracy')
plt.plot(epochs, val_acc_history, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()
# 학습 및 검증 리콜 시각화
plt.figure(figsize=(12, 6))
plt.plot(epochs, train_recall_history, label='Train Recall')
plt.plot(epochs, val_recall_history, label='Validation Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()
plt.title('Training and Validation Recall')
plt.show()
# 모델 평가 및 혼동 행렬 계산 함수
def evaluate_model_and_plot_confusion_matrix(model, dataloader, class_names):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    # 혼동 행렬 계산
    cm = confusion_matrix(all_labels, all_preds)
    # 비율로 변환
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    # 혼동 행렬 시각화
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Normalized Confusion Matrix')
    plt.show()
# 클래스 이름 설정
class_names = ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']
# # 검증 데이터셋에 대해 혼동 행렬 계산 및 시각화
# evaluate_model_and_plot_confusion_matrix(model, dataloaders['val'], class_names)

D:\ANACONDA3_ENVS\DR\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\ANACONDA3_ENVS\DR\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Training with learning rate: 0.0001, weight decay: 5e-05, dropout: 0.3
Epoch 0/29
train Loss: 1.8577 Acc: 0.3357 Recall: 0.3422
val Loss: 1.5991 Acc: 0.4926 Recall: 0.4901

Epoch 1/29
train Loss: 1.4228 Acc: 0.5126 Recall: 0.5173
val Loss: 1.1339 Acc: 0.6137 Recall: 0.6064

Epoch 2/29
train Loss: 1.1768 Acc: 0.6017 Recall: 0.6034
val Loss: 0.9658 Acc: 0.6751 Recall: 0.6774

Epoch 3/29
train Loss: 0.9850 Acc: 0.6692 Recall: 0.6693
val Loss: 0.8205 Acc: 0.7250 Recall: 0.7198

Epoch 4/29
train Loss: 0.8392 Acc: 0.7233 Recall: 0.7226
val Loss: 0.6949 Acc: 0.7724 Recall: 0.7755

Epoch 5/29
train Loss: 0.7175 Acc: 0.7620 Recall: 0.7605
val Loss: 0.6791 Acc: 0.7783 Recall: 0.7800

Epoch 6/29
train Loss: 0.6221 Acc: 0.7941 Recall: 0.7924
val Loss: 0.5993 Acc: 0.8029 Recall: 0.8058

Epoch 7/29
train Loss: 0.5373 Acc: 0.8217 Recall: 0.8201
val Loss: 0.5405 Acc: 0.8292 Recall: 0.8270

Epoch 8/29
train Loss: 0.4721 Acc: 0.8435 Recall: 0.8418
val Loss: 0.5202 Acc: 0.8382 Recall: 0.8352

Epoch 9/29


D:\ANACONDA3_ENVS\DR\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\ANACONDA3_ENVS\DR\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


train Loss: 1.8981 Acc: 0.3140 Recall: 0.3182
val Loss: 1.5496 Acc: 0.4860 Recall: 0.4855

Epoch 1/29
train Loss: 1.4629 Acc: 0.4963 Recall: 0.4998
val Loss: 1.1891 Acc: 0.6053 Recall: 0.6046

Epoch 2/29
train Loss: 1.1814 Acc: 0.6016 Recall: 0.6018
val Loss: 0.9232 Acc: 0.6932 Recall: 0.6937

Epoch 3/29
train Loss: 0.9727 Acc: 0.6779 Recall: 0.6772
val Loss: 0.7789 Acc: 0.7475 Recall: 0.7445

Epoch 4/29
train Loss: 0.8316 Acc: 0.7247 Recall: 0.7234
val Loss: 0.6770 Acc: 0.7799 Recall: 0.7753

Epoch 5/29
train Loss: 0.7054 Acc: 0.7649 Recall: 0.7633
val Loss: 0.6030 Acc: 0.8059 Recall: 0.8056

Epoch 6/29
train Loss: 0.6090 Acc: 0.7976 Recall: 0.7952
val Loss: 0.5695 Acc: 0.8162 Recall: 0.8167

Epoch 7/29
train Loss: 0.5325 Acc: 0.8239 Recall: 0.8221
val Loss: 0.5381 Acc: 0.8285 Recall: 0.8252

Epoch 8/29
train Loss: 0.4638 Acc: 0.8460 Recall: 0.8439
val Loss: 0.4972 Acc: 0.8429 Recall: 0.8393

Epoch 9/29
train Loss: 0.4032 Acc: 0.8667 Recall: 0.8647
val Loss: 0.4902 Acc: 0.8454 Recall:

D:\ANACONDA3_ENVS\DR\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\ANACONDA3_ENVS\DR\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


...

train Loss: 1.7589 Acc: 0.3791 Recall: 0.3853
val Loss: 1.3464 Acc: 0.5498 Recall: 0.5612

Epoch 1/29
train Loss: 1.2743 Acc: 0.5722 Recall: 0.5747
val Loss: 1.0159 Acc: 0.6620 Recall: 0.6606

Epoch 2/29
train Loss: 0.9949 Acc: 0.6690 Recall: 0.6679
val Loss: 0.7769 Acc: 0.7447 Recall: 0.7426

Epoch 3/29
train Loss: 0.8143 Acc: 0.7331 Recall: 0.7306
val Loss: 0.6377 Acc: 0.7919 Recall: 0.7886

Epoch 4/29
train Loss: 0.6793 Acc: 0.7776 Recall: 0.7754
val Loss: 0.6214 Acc: 0.7958 Recall: 0.8011

Epoch 5/29
train Loss: 0.5772 Acc: 0.8122 Recall: 0.8099
val Loss: 0.5197 Acc: 0.8258 Recall: 0.8285

Epoch 6/29
train Loss: 0.4996 Acc: 0.8372 Recall: 0.8342
val Loss: 0.4715 Acc: 0.8520 Recall: 0.8530

Epoch 7/29
train Loss: 0.4398 Acc: 0.8555 Recall: 0.8533
val Loss: 0.4600 Acc: 0.8532 Recall: 0.8502

Epoch 8/29
train Loss: 0.3876 Acc: 0.8730 Recall: 0.8710
val Loss: 0.3946 Acc: 0.8705 Recall: 0.8671

Epoch 9/29
train Loss: 0.3418 Acc: 0.8861 Recall: 0.8843
val Loss: 0.3934 Acc: 0.8757 Recall:

D:\ANACONDA3_ENVS\DR\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\ANACONDA3_ENVS\DR\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


train Loss: 1.7782 Acc: 0.3709 Recall: 0.3765
val Loss: 1.4247 Acc: 0.5282 Recall: 0.5324

Epoch 1/29
train Loss: 1.2846 Acc: 0.5619 Recall: 0.5638
val Loss: 0.9764 Acc: 0.6726 Recall: 0.6619

Epoch 2/29
train Loss: 1.0061 Acc: 0.6691 Recall: 0.6671
val Loss: 0.8144 Acc: 0.7297 Recall: 0.7351

Epoch 3/29
train Loss: 0.8247 Acc: 0.7293 Recall: 0.7271
val Loss: 0.6172 Acc: 0.7960 Recall: 0.7913

Epoch 4/29
train Loss: 0.6889 Acc: 0.7757 Recall: 0.7732
val Loss: 0.5746 Acc: 0.8132 Recall: 0.8139

Epoch 5/29
train Loss: 0.5919 Acc: 0.8056 Recall: 0.8030
val Loss: 0.5389 Acc: 0.8228 Recall: 0.8136

Epoch 6/29
train Loss: 0.5112 Acc: 0.8365 Recall: 0.8334
val Loss: 0.4298 Acc: 0.8627 Recall: 0.8624

Epoch 7/29
train Loss: 0.4463 Acc: 0.8543 Recall: 0.8522
val Loss: 0.4726 Acc: 0.8487 Recall: 0.8482

Epoch 8/29
train Loss: 0.3984 Acc: 0.8704 Recall: 0.8681
val Loss: 0.4268 Acc: 0.8648 Recall: 0.8642

Epoch 9/29
train Loss: 0.3534 Acc: 0.8837 Recall: 0.8816
val Loss: 0.3838 Acc: 0.8778 Recall:

D:\ANACONDA3_ENVS\DR\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\ANACONDA3_ENVS\DR\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


train Loss: 1.7957 Acc: 0.3683 Recall: 0.3724
val Loss: 1.8532 Acc: 0.4655 Recall: 0.4542

Epoch 1/29
train Loss: 1.4229 Acc: 0.5147 Recall: 0.5159
val Loss: 1.3984 Acc: 0.5790 Recall: 0.5718

Epoch 2/29
